In [1]:
# Importamos las librerias para el funcionamiento del codigo
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import seaborn as sns
import math


from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_absolute_percentage_error

import torch
from torch import nn, optim
import torch.utils.data as data
from torch.optim.lr_scheduler import ReduceLROnPlateau

import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# Guardar la variable imputation_lluvia_aitsu en otra variable para las modificaciones
variable = 'NIVEL_RIO_Aitzu-Urola'
data_river_2 = pd.read_csv('db_21.csv')
data_river_2 = data_river_2.set_index('Fecha')
# Asignar un formato de fecha a la columna fecha
data_river_2.index = pd.to_datetime(data_river_2.index, format='%Y-%m-%d %H:%M:%S')
# Se trabajara sobre la variable de nivel del rio Aitzu-Urola 
imputation_lluvia_aitsu = data_river_2.copy()
date_init = '1999-03-16 17:00'
date_end = '2023-09-30 23:00'
imputation_lluvia_aitsu = imputation_lluvia_aitsu[date_init:date_end] 

imputation_lluvia_aitsu.drop(['LLUVIA_Aitzu-Urola', 'NIVEL_RIO_Aitzu-Urola','CAUDAL_RIO_Aitzu-Urola'], axis=1, inplace=True)

In [7]:
#Definimos las siguientes funciones para el preprocesamiento de los datos
#Con la siguiente funciona nos aseguramos que los datos de entrada tengan el formato correcto para el funcionamiento de las redes recurrentes 
def create_sequences(data, lookback):
    xs = []
    ys = []

    for i in range(len(data)-lookback):
        #Mediante la variable lookback se define el numero de pasos temporales que se tomaran en cuenta para predecir el siguiente valor 
        #Se toman los valores de la serie temporal desde i hasta i+lookback 
        feature = data[i:(i+lookback)]
        target = data[i+lookback]
        # Los guardamos en las listas xs e ys 
        xs.append(feature)
        ys.append(target)
    #Convertimos las listas en tensores de pytorch  
    return torch.tensor(np.array(xs)), torch.tensor(np.array(ys)).unsqueeze(-1)

def find_nan_sequences(df, max_consecutive_nans=120, view=False):
    nan_index = df[df.isnull().any(axis=1)].index
    # print(nan_index)
    nan_sequences = []
    
    if len(nan_index) == 0:
        return nan_sequences

    start_date = nan_index[0]
    prev_date = nan_index[0]
    
    for i in range(1, len(nan_index)):
        current_date = nan_index[i]
        
        if current_date - prev_date != pd.Timedelta('1 hour 00:00:00'):
            # No es consecutivo, guardar la secuencia anterior
            end_date = prev_date
            num_nans = (end_date - start_date).total_seconds() / 3600 + 1
            
            if num_nans == 1:
                nan_sequences.append({
                    'type': 'single',
                    'start_date': start_date,
                    'end_date': end_date,
                    'num_nans': num_nans
                })
            elif num_nans <= max_consecutive_nans:
                nan_sequences.append({
                    'type': 'consecutive',
                    'start_date': start_date,
                    'end_date': end_date,
                    'num_nans': num_nans
                })
            elif num_nans > max_consecutive_nans:
                nan_sequences.append({
                    'type': 'long_consecutive',
                    'start_date': start_date,
                    'end_date': end_date,
                    'num_nans': num_nans
                })
            else:
                if view:
                    print(f'Secuencia de NaN mayor a {max_consecutive_nans} horas: {start_date} a {end_date}, {num_nans} valores')
            
            # Iniciar una nueva secuencia
            start_date = current_date
        
        prev_date = current_date
    
    # Manejar la última secuencia
    end_date = prev_date
    num_nans = (end_date - start_date).total_seconds() / 3600 + 1
    
    if num_nans == 1:
        nan_sequences.append({
            'type': 'single',
            'date': start_date
        })
    elif num_nans <= max_consecutive_nans:
        nan_sequences.append({
            'type': 'consecutive',
            'start_date': start_date,
            'end_date': end_date,
            'num_nans': num_nans
        })
    else:
        if view:
            print(f'Secuencia de NaN mayor a {max_consecutive_nans} horas: {start_date} a {end_date}, {num_nans} valores')
    
    return nan_sequences

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def predict_future(model, last_known_sequence, num_future_steps):
    model.eval()
    future_predictions = []
    current_sequence = last_known_sequence.clone().to(device)

    for _ in range(num_future_steps):
        with torch.no_grad():
            prediction = model(current_sequence.float())
            future_predictions.append(prediction[0, -1, 0].item())
            current_sequence = torch.cat((current_sequence[:, 1:, :], prediction[:, -1:, :]), dim=1)

    return future_predictions

def calculate_metrics(y_true, y_pred):
    mse = ((y_true - y_pred) ** 2).mean()
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    return rmse, mae, mape


In [14]:
from sklearn.impute import KNNImputer

# Inicializar el imputador KNN
imputer = KNNImputer(n_neighbors=48)

# Realizar la imputación
imp_all_variables = pd.DataFrame(imputer.fit_transform(imputation_lluvia_aitsu), columns=imputation_lluvia_aitsu.columns)
print("Imputación con KNN:\n", imp_all_variables)



Imputación con KNN:
         LLUVIA_Aitzu  NIVEL_RIO_Aitzu  CAUDAL_RIO_Aitzu  LLUVIA_Aizarnazabal  \
0           0.000000            0.260            1.1905                  0.0   
1           0.000000            0.260            1.1905                  0.0   
2           0.000000            0.260            1.1905                  0.0   
3           0.000000            0.260            1.1905                  0.0   
4           0.004167            0.260            1.1905                  0.0   
...              ...              ...               ...                  ...   
215138      0.000000            0.120            0.1553                  0.0   
215139      0.000000            0.120            0.1553                  0.0   
215140      0.000000            0.118            0.1489                  0.0   
215141      0.000000            0.120            0.1553                  0.0   
215142      0.000000            0.118            0.1489                  0.0   

        NIVEL_RIO_

In [15]:
imp_all_variables.to_csv('imputed_KNN_db_21.csv', index=True)

In [16]:
print(imp_all_variables.isna().sum())

LLUVIA_Aitzu                      0
NIVEL_RIO_Aitzu                   0
CAUDAL_RIO_Aitzu                  0
LLUVIA_Aizarnazabal               0
NIVEL_RIO_Aizarnazabal            0
CAUDAL_RIO_Aizarnazabal           0
LLUVIA_Antzuola                   0
LLUVIA_Arantzazu                  0
LLUVIA_Araotz                     0
LLUVIA_Arkaka                     0
LLUVIA_Barrendiola                0
NIVEL_RIO_Barrendiola             0
CAUDAL_RIO_Barrendiola            0
LLUVIA_Barrendiola_Pluviometro    0
LLUVIA_Elosua                     0
LLUVIA_Erdoizta                   0
LLUVIA_Ibaieder                   0
NIVEL_RIO_Ibaieder                0
CAUDAL_RIO_Ibaieder               0
LLUVIA_Lareo                      0
LLUVIA_Lastur                     0
LLUVIA_Laurgain                   0
LLUVIA_Matxinbenta                0
NIVEL_RIO_Matxinbenta             0
CAUDAL_RIO_Matxinbenta            0
LLUVIA_Troya                      0
LLUVIA_zumarraga                  0
LLUVIA_azpeitia             

In [17]:
#standar scaler data 

